# hCassandra runTests

## Description

---

### Objective

   This test aims to **automate**:
   
   (1) The execution of the Hydra Cassandra Stress Test (hCassandra) for increasing client load.
   
   (2) The generation of performance results presented in the form of tables and graphs for relevant metrics. 
   
   To this end, performance of the Cassandra Cluster is measured as the number of clients writing and reading into the DataBase is increased. The number of clients can be defined by the user. 
   
 
### Customize the Test

   Modify **total_num_clients** to change the sets of clients for which you wish to execute the test.
   
   Current tests have been run for a maximum of **10000** clients and a duration of 5 minutes against a 3-node Cluster (for further details on Software & Hardware specs please refer to the *Software & Hardware Specs* section).
   
### Useful HINTS for running the test

- If test has been previously executed and output is still shown, you can restart (delete former results) by selecting in the top menu Cell -> All Output -> Clear
- To run test, step on top of the code cells and press the 'run cell' button on the top menu. For automatic Run select from the top menu Cell -> Run All
- If you wish to store your results. After RUN is finished, generate your own report by selecting FILE -> Download as -> Markdown (.md) (or any other preferred format).

## Software & Hardware Specs

---

The tests were executed on Google Cloud Servers, with the following specs:

#### Cassandra Cluster

- 3 Node Cluster, each with the following specs:
   - 16 vCPUs
   - RAM: 60 GB
   - Disk: 60 GB
   - OS: Debian 3.16.7-ckt25-2
  
- Cassandra + Cassandra-Tools Version: 3.0.6

#### Hydra Cluster

- **MASTER**: 1 Server
   - 4 vCPUs
   - RAM: 15 GB
   - OS: Ubuntu 14.04

- **SLAVES**: 9 Servers (hosts to the cassandra-stress tool)
   - 16 vCPUs 
   - RAM: 60 GB
   - Disk: 60 GB
   - OS: Debian 3.16.7-ckt25-2
   
### Important 

- For the performance tests maximum file open limit (ulimit) had to be increased for the Master Node.

---

## hCassandra Test 1: Fixed Number of Stress Clients (Debug Mode) 

---

The following test runs a SINGLE execution of the Cassandra Test for a fixed number of clients (total_client_count) and operations (total_ops_count). Runs in debug mode: showing logger info during execution. 

In [ ]:
!python ./src/hCassandra_test.py --cluster_ips='10.10.0.155,10.10.2.218,10.10.3.116' --total_client_count=8000 --total_ops_count=2000

---

## hCassandra Test 2: Increasing the Number of stress clients (multiple runs)

---


**IMPORTANT**:

   If you want to change the number of clients and/or number of operations for your test, please set values to desired in the following section:


In [1]:
# Define num Client(s) / Operation(s) 
total_num_clients = [10, 100, 500, 1000, 3000, 4000, 6000, 8000, 10000]
total_ops_count = [1000000]
# Set IPs of Nodes in Cassandra Cluster
cassandra_cluster_ips = '10.10.0.155,10.10.2.218,10.10.3.116'

**UTIL FUNCTIONS**

In [2]:
import json
import ast

def get_result(test_stdout):
    """This Function gets (filters) the Cassandra Test Results from stdout"""
    index_start = test_stdout.find('Cassandra Stress Results: \n')
    index_end = test_stdout.find('Calling Server shutdown')
    if index_start != -1:
        results = test_stdout[(index_start + len('Cassandra Stress Results: \n')):index_end]
        res_dict = ast.literal_eval(results)
        return res_dict
    else:
        return {}

The following block of code is the actual **EXECUTION OF THE CASSANDRA SCALE TESTS**. This may take a couple of minutes:

In [3]:
import subprocess
import os
import json
import signal

hCassandra_results = dict()

print 'STARTING CASSANDRA STRESS TESTS \n'
# Execute hCassandra_test for given client_count
for idx1, clients in enumerate(total_num_clients):
    for idx2, ops in enumerate(total_ops_count):
        print ('Test (%s/%s) in progress.. Please wait until test is completed..' % ((len(total_ops_count) * idx1) + idx2 + 1,len(total_num_clients) * len(total_ops_count)))
        # Execute hCassandra_test.py (python script for hCassandra Scale Test)
        hcass_cmd = "python ./src/hCassandra_test.py --cluster_ips=%s --total_client_count=%s --total_ops_count=%s" % (cassandra_cluster_ips, clients, ops)
        stress_test = subprocess.Popen(hcass_cmd, stdout=subprocess.PIPE,
                                          stderr=subprocess.PIPE, shell=True, preexec_fn=os.setsid)
        stdout, stderr = stress_test.communicate()
        results_dict = get_result(stdout)
        if len(results_dict) <= 1:
            print ('There was an ERROR while attempting to parse stdout...')
            print 'STDOUT: %s' % stdout
            print 'STDERR: %s' % stderr
        if not str(clients) in hCassandra_results:
            hCassandra_results[str(clients)] = dict()
        hCassandra_results[str(clients)][str(ops)] = results_dict
        print 'Test SUCCESFULLY completed... \n'

print 'END OF TESTS:'
print 'ALL TESTS HAVE BEEN COMPLETED. PLEASE PROCEED TO GENERATE GRAPHS & TABLES WITH PERFORMANCE RESULTS.'

STARTING CASSANDRA STRESS TESTS 

Test (1/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (2/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (3/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (4/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (5/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (6/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (7/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (8/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

Test (9/9) in progress.. Please wait until test is completed..
Test SUCCESFULLY completed... 

END OF TESTS:
ALL TESTS HAVE BEEN COMPLETED. PLEASE PROCEED TO GENERATE GRAPHS & TABLES WITH PERFORMANCE RESULT

  **NOTE:**
  
  ---
   
   Wait until RESULTS (**hCassandra_results**) are generated for all cases, and then execute the following blocks to generate:
   (1) Tables with results (markdown compatible) and 
   (2) Graphs.
   
   The **END OF TEST** is indicated by a message. Please wait...
   
   ---

### RESULT PROCESSING & TABLE/ GRAPH GENERATION

---

In this section, we process the results for generating tables with performance values and graphs that reflect number of operations per second and median latency for increased number of clients. 

**NOTE**
If you are interested in representing any other performance metric, follow the pattern followed for any of the two graps already provided. 

In [4]:
class ListTable(list):
    """ Overridden list class which takes a 2-dimensional list of 
        the form [[1,2,3],[4,5,6]], and renders an HTML Table in 
        IPython Notebook. """
    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")
            
            for col in row:
                html.append("<td>{0}</td>".format(col))
            
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [5]:
results_per_ops = dict()

# Table Format: Metrics
header = [
            '# Clients',
            'total_ops',
            'op/s',
            'pk/s',
            'med',
            '.95',
            '.99',
            'max',
            'max_ms',
            'sdv_ms',
            'op_time'
        ]

data_matrix_write = ListTable()
data_matrix_read = ListTable()

data_matrix_write.append(header)
data_matrix_read.append(header)

for ops in total_ops_count:
    results_per_ops[str(ops)] = dict()
    for clients in total_num_clients:
        if str(clients) in hCassandra_results:
            res_dict = ast.literal_eval(hCassandra_results[str(clients)][str(ops)])
            results_per_ops[str(ops)][str(clients)] = res_dict
            data_matrix_write.append([clients, res_dict['write']['total ops'], res_dict['write']['op/s'], res_dict['write']['pk/s'], res_dict['write']['med'], res_dict['write']['.95'], res_dict['write']['.99'], res_dict['write']['max'], res_dict['write']['max_ms'], res_dict['write']['sdv_ms'], res_dict['write']['op_time']])
            data_matrix_read.append([clients, res_dict['read']['total ops'], res_dict['read']['op/s'], res_dict['read']['pk/s'], res_dict['read']['med'], res_dict['read']['.95'], res_dict['read']['.99'], res_dict['read']['max'], res_dict['read']['max_ms'], res_dict['read']['sdv_ms'], res_dict['read']['op_time']])

### Result Generation: Table Format

Next, results are displayed in a Table, following the markdown format..  



The next table represents the results for the **WRITE** Operations:

---

*Table 1. "Cassandra Performance over WRITE Operation."*

In [6]:
data_matrix_write

[['# Clients',
  'total_ops',
  'op/s',
  'pk/s',
  'med',
  '.95',
  '.99',
  'max',
  'max_ms',
  'sdv_ms',
  'op_time'],
 [10,
  [5771705],
  [19239],
  [19239],
  [0.5],
  [0.6],
  [0.7],
  [117.3],
  [],
  [0.0],
  [' 00:05:00']],
 [100,
  [7833430, 6863609, 7691190, 7365064, 7756814],
  [26111, 22879, 25637, 24550, 25856],
  [26111, 22879, 25637, 24550, 25856],
  [0.6, 0.7, 0.6, 0.6, 0.6],
  [1.1, 1.3, 1.1, 1.2, 1.1],
  [1.8, 2.0, 1.7, 1.9, 1.7],
  [141.7, 142.0, 142.3, 142.0, 170.3],
  [],
  [0.0, 0.0, 0.0, 0.0, 0.0],
  [' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00', ' 00:05:00']],
 [500,
  [2722894,
   2741985,
   2712670,
   2831352,
   2709076,
   2729771,
   2781676,
   2731553,
   2725194,
   2699985,
   2727535,
   2690215,
   2814241,
   2783465,
   2853107,
   2754434,
   2777226,
   2785832,
   2723791,
   2745371,
   2762452,
   2786146,
   2797024,
   2749165,
   2725674],
  [9076,
   9139,
   9042,
   9438,
   9030,
   9099,
   9272,
   9105,
   9084,
   9000,
   9092,
   8967,
   9381,
   9278,
   9510,
   9181,
   9257,
   9286,
   9079,
   9151,
   9207,
   9287,
   9323,
   9164,
   9085],
  [9076,
   9139,
   9042,
   9438,
   9030,
   9099,
   9272,
   9105,
   9084,
   9000,
   9092,
   8967,
   9381,
   9278,
   9510,
   9181,
   9257,
   9286,
   9079,
   9151,
   9207,
   9287,
   9323,
   9164,
   9085],
  [1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.3,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2,
   1.2],
  [5.1,
   5.0,
   4.9,
   4.7,
   5.0,
   5.0,
   5.1,
   5.0,
   5.0,
   5.0,
   4.9,
   5.2,
   4.8,
   4.9,
   4.9,
   5.0,
   4.9,
   4.9,
   4.9,
   4.8,
   4.9,
   4.9,
   4.9,
   4.8,
   5.1],
  [11.6,
   11.6,
   10.9,
   10.9,
   11.3,
   11.6,
   11.9,
   12.1,
   12.0,
   11.8,
   10.6,
   12.5,
   10.7,
   11.0,
   10.8,
   11.8,
   11.0,
   11.5,
   11.2,
   10.8,
   11.1,
   11.7,
   11.5,
   10.5,
   11.7],
  [400.7,
   212.8,
   446.4,
   259.7,
   446.5,
   266.3,
   220.2,
   347.1,
   233.4,
   477.7,
   201.2,
   535.4,
   396.4,
   351.1,
   606.8,
   607.8,
   338.2,
   606.6,
   297.9,
   431.2,
   327.9,
   496.4,
   251.5,
   440.9,
   201.4],
  [],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:04:59',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00',
   ' 00:05:00']],
 [1000,
  [1608274,
   1554101,
   1638563,
   1622628,
   1604470,
   1596125,
   1597535,
   1614033,
   1582393,
   1617317,
   1574693,
   1632109,
   1552880,
   1588393,
   1582649,
   1607128,
   1630516,
   1590039,
   1547883,
   1598059,
   1590931,
   1596257,
   1602769,
   1551583,
   1586748,
   1549799,
   1576310,
   1559120,
   1620122,
   1580673,
   1565673,
   1561645,
   1569945,
   1613719,
   1633922,
   1564720,
   1597356,
   1558883,
   1567870,
   1563257,
   1552078,
   1638551,
   1560872,
   1566894,
   1601146,
   1611668,
   1562828,
   1583051,
   1586614,
   1570978],
  [5361,
   5180,
   5461,
   5409,
   5348,
   5320,
   5325,
   5380,
   5275,
   5391,
   5249,
   5440,
   5176,
   5295,
   5275,
   5357,
   5435,
   5300,
   5160,
   5327,
   5303,
   5321,
   5342,
   5172,
   5289,
   5165,
   5254,
   5197,
   5400,
   5269,
   5219,
   5205,
   5233,
   5379,
   5446,
   5216,
   5324,
   5196,
   5226,
   5211,
   5173,
   5462,
   5202,
   5223,
   5337,
   5372,
   5209,
   5276,
   5289,
   5237],
  [5361,
   5180,
   5461,
   5409,
   5348,
   5320,
   5325,
   5380,
   5275,
   5391,
 

The next table represents the results for the **READ** Operations:

---

*Table 1. "Cassandra Performance over READ Operation."*

In [7]:
data_matrix_read

# Clients,total_ops,op/s,pk/s,med,.95,.99,max,max_ms,sdv_ms,op_time
10,[2387590],[19896],[19896],[0.5],[0.6],[0.7],[50.7],[],[0.0],[' 00:02:00']
100,"[2966491, 2548810, 2981868, 2963920, 2663444]","[24721, 21240, 24849, 24699, 22195]","[24721, 21240, 24849, 24699, 22195]","[0.7, 0.8, 0.7, 0.7, 0.7]","[1.3, 1.5, 1.2, 1.2, 1.5]","[1.9, 2.4, 2.0, 1.9, 2.2]","[41.7, 44.9, 44.7, 41.9, 45.9]",[],"[0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00']"
500,"[924466, 786641, 822773, 798311, 812672, 883574, 870990, 826969, 833749, 853942, 820474, 850336, 859497, 839913, 805788, 825712, 869881, 853224, 773938, 779279, 811286, 815335, 821199, 813634, 862179]","[7704, 6555, 6855, 6651, 6772, 7363, 7258, 6891, 6948, 7116, 6835, 7085, 7162, 6997, 6714, 6879, 7249, 7109, 6448, 6493, 6761, 6794, 6843, 6780, 7184]","[7704, 6555, 6855, 6651, 6772, 7363, 7258, 6891, 6948, 7116, 6835, 7085, 7162, 6997, 6714, 6879, 7249, 7109, 6448, 6493, 6761, 6794, 6843, 6780, 7184]","[1.4, 1.6, 1.5, 1.6, 1.5, 1.4, 1.4, 1.5, 1.5, 1.5, 1.5, 1.4, 1.4, 1.5, 1.5, 1.5, 1.4, 1.5, 1.6, 1.6, 1.5, 1.5, 1.5, 1.5, 1.5]","[6.6, 7.8, 7.3, 7.7, 7.4, 6.9, 7.2, 7.7, 7.6, 7.1, 7.6, 7.2, 7.2, 7.4, 8.0, 7.5, 7.2, 7.2, 8.1, 7.7, 7.7, 7.8, 7.7, 7.9, 7.3]","[15.5, 20.0, 19.6, 20.7, 19.2, 17.4, 18.6, 20.2, 19.2, 18.6, 19.6, 17.9, 17.9, 20.3, 19.9, 19.2, 18.8, 19.6, 20.8, 18.7, 19.8, 19.9, 20.6, 20.6, 18.7]","[146.1, 122.9, 167.1, 137.7, 170.7, 166.1, 125.3, 133.1, 199.9, 159.7, 127.7, 141.5, 138.1, 151.5, 139.3, 203.2, 170.9, 143.2, 141.7, 137.4, 124.0, 139.0, 203.2, 138.1, 152.6]",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00', ' 00:02:00']"
1000,"[539333, 529886, 526751, 561166, 530299, 563793, 572017, 583467, 556249, 518916, 542327, 528026, 534972, 555211, 530342, 538619, 580670, 557625, 542828, 545477, 579177, 545260, 530234, 520245, 552400, 530813, 535367, 525793, 552322, 559614, 556948, 543721, 534775, 521566, 551549, 539219, 518726, 536852, 531561, 531748, 533341, 536202, 517842, 563184, 528924, 559263, 557886, 522275, 570862, 566658]","[4494, 4415, 4389, 4676, 4419, 4698, 4767, 4862, 4635, 4322, 4519, 4399, 4457, 4626, 4419, 4488, 4839, 4647, 4523, 4545, 4826, 4543, 4418, 4334, 4603, 4423, 4461, 4380, 4603, 4663, 4641, 4530, 4456, 4345, 4596, 4493, 4321, 4473, 4429, 4430, 4444, 4468, 4315, 4693, 4407, 4660, 4649, 4352, 4757, 4722]","[4494, 4415, 4389, 4676, 4419, 4698, 4767, 4862, 4635, 4322, 4519, 4399, 4457, 4626, 4419, 4488, 4839, 4647, 4523, 4545, 4826, 4543, 4418, 4334, 4603, 4423, 4461, 4380, 4603, 4663, 4641, 4530, 4456, 4345, 4596, 4493, 4321, 4473, 4429, 4430, 4444, 4468, 4315, 4693, 4407, 4660, 4649, 4352, 4757, 4722]","[2.4, 2.5, 2.4, 2.3, 2.4, 2.2, 2.2, 2.2, 2.3, 2.5, 2.4, 2.4, 2.4, 2.3, 2.4, 2.4, 2.2, 2.3, 2.4, 2.4, 2.2, 2.3, 2.4, 2.5, 2.4, 2.4, 2.4, 2.5, 2.3, 2.3, 2.3, 2.4, 2.4, 2.5, 2.3, 2.4, 2.5, 2.4, 2.4, 2.4, 2.4, 2.4, 2.5, 2.3, 2.4, 2.3, 2.2, 2.5, 2.2, 2.2]","[12.2, 12.4, 12.2, 12.0, 12.1, 11.4, 11.5, 10.9, 11.6, 12.6, 11.5, 12.1, 11.7, 12.0, 11.8, 11.9, 11.5, 11.3, 11.9, 11.7, 11.4, 11.8, 12.2, 12.1, 11.7, 12.0, 12.0, 11.8, 11.6, 12.3, 12.0, 11.9, 12.0, 12.2, 11.5, 11.9, 12.4, 11.7, 12.0, 11.9, 12.3, 12.1, 12.1, 11.3, 12.0, 11.4, 11.4, 12.4, 11.2, 11.6]","[28.8, 30.8, 29.5, 30.2, 28.4, 27.9, 28.0, 24.4, 28.0, 28.6, 26.2, 28.7, 28.1, 30.4, 28.5, 28.8, 28.0, 26.5, 29.3, 27.6, 26.0, 27.1, 29.5, 29.7, 27.5, 28.1, 29.0, 27.6, 28.2, 29.7, 28.7, 28.3, 28.2, 29.5, 28.3, 29.8, 30.5, 27.8, 28.2, 29.3, 28.2, 27.8, 27.2, 26.5, 27.3, 28.5, 26.1, 28.0, 26.5, 27.7]","[169.3, 166.5, 171.4, 182.6, 203.6, 208.6, 196.2, 209.2, 186.1, 187.5, 253.3, 

### Result Generation: Graphs

Next, results are displayed in Graphs. 

--- 

**IMPORTANT**

Please, MODIFY the graphs name in order to avoid OVERWRITING previous results.



---

In [8]:
ops_second_graph_filename = "hCassandra_ops_sec_006"
median_latency_graph_filename = "hCassandra_med_006"

In [12]:
%%capture

import plotly.plotly as py
from plotly.graph_objs import *
import operator
import numpy

data_matrix = [['# ops', '# Clients', 'total_ops', 'op/s', 'pk/s', 'med', '.95', '.99', 'max', 'max_ms', 'sdv_ms', 'op_time']]


traces_plot1 = []
traces_plot2 = []

# For each trace = client count
for ops_count, tests_per_trace in results_per_ops.iteritems():
    
    total_ops = []
    op_s = []
    op_s_r = []
    pk_s = []
    med = []
    med_r = []
    p95 = []
    p99 = []
    max_lat = []
    max_ms = []
    sdv_ms = []
    op_time = []
    
    for num_clients, res_dict in tests_per_trace.iteritems():
        op_s.append(sum((ops for ops in res_dict['write']['op/s'])))
        med.append(numpy.median(res_dict['write']['med']))
        op_s_r.append(sum((ops for ops in res_dict['read']['op/s'])))
        med_r.append(numpy.median(res_dict['read']['med']))
        # Un-Comment for any other feature you want to depict...
        
        #total_ops.append(sum((ops for ops in res_dict['write']['total ops'])))
        #pk_s.append(res_dict['write']['pk/s'])
        #p95.append(res_dict['write']['.95'])
        #p99.append(res_dict['write']['.99'])
        #max_lat.append(res_dict['write']['max'])
        #max_ms.append(res_dict['write']['max_ms'])
        #sdv_ms.append(res_dict['write']['sdv_ms'])
        #op_time.append(res_dict['write']['op_time'])
        
        
    trace_plot1 = Scatter(
          x=total_num_clients,
          y=op_s, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[WRITE]',
          line=dict(
            shape='spline'
            )
        )
    
    trace_plot2 = Scatter(
          x=total_num_clients,
          y=op_s_r, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[READ]',
          line=dict(
            shape='spline'
            )
        )
        
    trace_plot3 = Scatter(
          x=total_num_clients,
          y=med, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[WRITE]', 
          line=dict(
            shape='spline'
            )
        )

    trace_plot4 = Scatter(
          x=total_num_clients,
          y=med_r, 
          mode = 'lines+markers',
          name = 'ops_count = ' + str(ops_count) + '[READ]', 
          line=dict(
            shape='spline'
            )
        )
    
    traces_plot1.append(trace_plot1)
    #traces_plot1.append(trace_plot2)
    traces_plot2.append(trace_plot3)
    #traces_plot2.append(trace_plot4)


### Result Generation: operations per second vs. client count

The following graph illustrates how, the number of operations per second changes while the number of clients increases 

In [13]:
%%capture plot_operations --no-stdout

data = Data(traces_plot1)
# Edit the layout
layout = dict(title = 'op/s vs. # Clients',
              xaxis = dict(title = '# clients'),
              yaxis = dict(title = 'op/s'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = ops_second_graph_filename)

### Result Generation: median latency vs. client count

The following graph illustrates median latency in miliseconds for each operation during that run as the number of clients increases. 

In [14]:
%%capture plot_med --no-stdout

data = Data(traces_plot2)
# Edit the layout
layout = dict(title = 'Median Latency vs. Client Count',
              xaxis = dict(title = '# Clients'),
              yaxis = dict(title = 'Median Latency [ms]'),
              )

# Plot and embed in notebook
fig = dict(data=data, layout=layout)
py.iplot(fig, filename = median_latency_graph_filename)

Overall, these benchmarks represent the **maximum throughput** of a 3 node cluster for the *default* model generated by the cassandra-stress tool. For accurate performance assessment of an application a range of parameters (including data model, queries, etc.) need to be adjusted. 